# 2. Зейін механизмы (attention mechanisms)
# 2. Механизм внимания (attention mechanisms)

1. Бұл блокнотта қолданылатын пакеттер:
2. Библиотеки, которые используются в этом блокноте:

In [57]:
from importlib.metadata import version

from sympy.polys.polyconfig import query

print("torch version:", version("torch"))

torch version: 2.5.1


- Бұл тарауда зейін механизмдеры қарастырылады
- В этой главе мы рассмотрим механизмы внимания.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="1000px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="1000px">

## 2.1 Ұзын cөз тізбектерін модельдеу мәселесі
## 2.1 Проблема моделирования длинных последовательностей

- Тілдерді сөзбе сөз дұрыс аудару мүмкін емес, себебі олардың грамматикалық құрылымдары өзгеше
***
- Дословный перевод (слово в слово) нецелесообразен 
из-за различий в грамматических структурах между исходным языком и языком перевода

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="1000px">

- Трансформер модельдер пайда болғанға дейін машиналық аударма міндеттері үшін энкодер-декодерлі RNN құрылымы жиі қолданылды.
- Бұл құрылымда `энкодер` бастапқы тілдегі токендер тізбегін өңдейді, ол бүкіл кіріс тізбегінің ықшамдалған көрінісін генерациялау үшін жасырын күйде — нейрондық желі аралық қабаттың бір түрін пайдаланады.

***

- До появления трансформерных моделей в задачах машинного перевода широко использовались RNN-сети с архитектурой «энкодер-декодер».

- В этой схеме `энкодер` обрабатывает последовательность токенов исходного языка, используя скрытое состояние — своего рода промежуточный слой нейронной сети — для создания сжатого представления всей входной последовательности:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="1000px">

## 2.2 Зейін механизмдері арқылы деректердегі тәуелділіктерді (заңдылықтарды) анықтау
## 2.2 Улавливание зависимостей (закономерностей) в данных с помощью механизмов внимания

- Зейін механизмі арқылы  декодер барлық кіріс токендеріне таңдамалы түрде қол жеткізе алады. Бұл нақты бір шығыс токенін генерациялау кезінде кейбір кіріс токендерінің басқаларға қарағанда маңыздырақ екенін білдіреді.
***
- С помощью механизма внимания декодер, генерирующий текст, способен избирательно обращаться ко всем входным токенам. Это означает, что при генерации определённого выходного токена некоторые входные токены имеют большее значение, чем другие.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="1000px">

- Трансформерлердегі өзіндік зейін (self-attention) — бұл тізбектегі әрбір 
позицияға сол тізбектің барлық басқа позицияларымен өзара әрекеттесуге 
және олардың маңыздылығын анықтауға мүмкіндік беру арқылы кіріс 
репрезентацияларын жақсартуға арналған техника.
***
- Само-внимание (self-attention) в трансформерах — это техника, предназначенная для улучшения входных представлений, позволяющая каждой позиции в последовательности взаимодействовать со всеми другими позициями в той же последовательности и определять их релевантность.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="1000px">

## 2.3 Өзіндік зейін (self-attention) арқылы кіріс деректердің әртүрлі бөліктеріне зейін аудару
## 2.3 Фокусировка на различных частях входных данных с помощью само-внимания (self-attention)

### 2.3.1 Үйретілетін салмақтарсыз қарапайым өзіндік зейін механизмі
### 2.3.1 Простой механизм само-внимания (self-attention) без обучаемых весов

- Бұл бөлімде ешқандай үйретілетін салмақтары жоқ өзіндік зейіннің (self-attention) өте қарапайым нұсқасы түсіндіріледі.
- Бұл тек көрнекілік үшін жасалған және трансформерлерде қолданылатын зейін механизмі ЕМЕС.
- Келесі 3.3.2-бөлімде осы қарапайым зейін механизмі нағыз өзіндік зейін механизмін іске асыру үшін кеңейтілетін болады.
- Бізге $x^{(1)}$-ден $x^{(T)}$ -ға дейінгі кіріс тізбегі берілді деп есептейік.
    - Кіріс деректер — 1-тарауда сипатталғандай, токен эмбеддингтеріне түрлендірілген мәтін (мысалы, "Your journey starts with one step" сияқты сөйлем).
    - Мысалы, $x^{(1)} - "Your" сөзін білдіретін d-өлшемді вектор, және т.б.
- **Мақсат**: $x^{(1)}$-ден $x^{(T)}$-ға дейінгі әрбір кіріс 
тізбегінің элементі $x^{(i)}$ үшін контекстік векторларды $z^{(i)}$ есептеу ($z$ мен $x$ кеңістік өлшемдері бірдей).
    - $z^{(i)}$ контекстік векторы — бұл $x^{(1)}$ -ден $x^{(T)}$ -ға дейінгі кіріс     деректерінің салмақталған қосындысы.
    - Контекстік вектор белгілі бір кіріс элементі үшін "контекстке" арнайы болады.
    - Кез келген кіріс токенінің орнына $x^{(i)}$ белгісін қолданудың орнына, екінші кіріс элементін, яғни $x^{(2)}$-ні қарастырайық.
    - Нақты мысалды жалғастыру үшін, $z^{(i)}$ белгісінің орнына екінші шығыс контекстік векторын, яғни $z^{(2)}$ -ні қарастырамыз.
    - Екінші контекстік вектор, z$z^{(2)}$  екінші кіріс элементіне, $x^{(2)}$ -ге қатысты салмақталған $x^{(1)}$-ден $x^{(T)}$-ға дейінгі барлық кіріс элементтерінің салмақталған қосындысы.
    - Зейін салмақтары — бұл $z^{(2)}$-ні есептеу кезінде әрбір кіріс элементінің салмақталған қосындыға қаншалықты үлес қосатынын анықтайтын салмақтар.
    - Қысқаша айтқанда, $z^{(2)}$-ні $x^{(2)}$-нің өзгертілген нұсқасы деп ойлаңыз, ол сонымен қатар қарастырылып отырған тапсырма үшін маңызды болып табылатын барлық басқа кіріс элементтері туралы ақпаратты қамтиды.

***

- В этом разделе объясняется очень упрощённый вариант само-внимания (self-attention), который не содержит обучаемых весов.
- Это сделано исключительно в иллюстративных целях и это НЕ тот механизм внимания, который используется в трансформерах.
- В следующем разделе, 3.3.2, этот простой механизм внимания будет расширен для реализации настоящего механизма self-attention.

- Предположим, нам дана входная последовательность от $x^{(1)}$ до $x^{(T)}$.
- Входные данные — это текст (например, предложение "Your journey starts with one step"), который уже был преобразован в токенные эмбеддинги, как описано в главе 2.
- Например, $x^{(1)}$ — это d-мерный вектор, представляющий слово "Your", и так далее.

- **Цель**: вычислить контекстные векторы $z^{(i)}$ для каждого элемента входной последовательности $x^{(i)}$ от $x^{(1)}$до $x^{(T)}$ (где $z$ и $x$ имеют одинаковую размерность).
    - Контекстный вектор $z^{(i)}$ это взвешенная сумма входных векторов от $x^{(1)}$до $x^{(T)}$.
    - Контекстный вектор является специфичным для "контекста" определённого входного элемента.
    - Вместо того чтобы использовать $x^{(i)}$ как обозначение для произвольного входного токена, давайте рассмотрим второй входной элемент, $x^{(2)}$.
    - И чтобы продолжить на конкретном примере, вместо $z^{(i)}$ мы рассмотрим второй выходной контекстный вектор, $z^{(2)}$.
    - Второй контекстный вектор, $z^{(2)}$  это взвешенная сумма всех входных векторов от $x^{(1)}$до $x^{(T)}$, взвешенная относительно второго входного элемента, $x^{(2)}$.
    - Веса внимания — это коэффициенты, которые определяют, какой вклад вносит каждый из входных элементов во взвешенную сумму при вычислении $z^{(2)}$.
    - Короче говоря, думайте о $z^{(2)}$ как о модифицированной версии $x^{(2)}$, которая также включает информацию обо всех других входных элементах, релевантных для текущей задачи.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="1000px">


- Шартты түрде, нормализациядан өтпеген зейін салмақтары `«зейін баллдары` (attention scores) деп аталады,    
ал қосындысы 1-ге тең болатын нормализациядан  өткен зейін бағалары `«зейін салмақтары»` (attention weights) деп аталады.

***

- Условно, ненормализованные веса внимания называют `«оценками внимания»` (attention scores),    
в то время как нормализованные оценки, сумма которых равна 1, называют `«весами внимания»` (attention weights).

- **1-қадам**: нормаланбаған $\omega$  зейін баллдарын есептеу
- Айталық, біз екінші кіріс токенін сұраныс ретінде қолданамыз, яғни, $q^{(2)} = x^{(2)}$, бұл жағдайда біз нормаланбаған зейін баллдарын скалярлық көбейтінділер арқылы есептейміз:
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- Жоғарыда, $\omega$ — нормаланбаған зейін баллдарын (attention scores) белгілеу үшін қолданылатын «омега» грек әрпі.
    - $\omega_{21}$ жазуындағы «21» төменгі индексі кіріс тізбегінің 2-ші элементі 1-ші элементіне қатысты сұраныс ретінде қолданылғанын білдіреді.


***

- **Шаг 1**: Вычислить ненормализованные оценки внимания $\omega$
- Предположим, мы используем второй входной токен в качестве запроса, то есть q^{(2)} = x^{(2)}$. 
- В этом случае мы вычисляем ненормализованные оценки внимания с помощью скалярных произведений:
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$

- Выше $\omega$ — это греческая буква «омега», которая используется для обозначения ненормализованных оценок внимания (attention scores).
    - Нижний индекс «21» в записи $\omega_{21}$ означает, что второй элемент входной последовательности использовался в качестве запроса к первому элементу.
***

- 3 өлшемді векторлар түрінде ұсынылған келесі кіріс сөйлем берілген деп алайық (мұнда көрнекілік үшін  өте шағын ендіру өлшемін қолданамыз):
- Предположим, у нас есть следующее входное предложение, уже представленное в виде 3-мерных векторов (для наглядности здесь используется очень малая размерность вложений):

- Жоғарыда көрсетілген тензордың әр қатары сөзді, ал әр баған ендіру (embedding) өлшемін білдіреді
- Бұл бөлімнің негізгі мақсаты — $x^{(2)}$ арқылы $z^{(2)}$ контекст векторының қалай есептелетінін көрсету.
- Төмендегі суретте — скалярлық көбейту операциясы арқылы $x^{(2)}$ мен барлық басқа кіріс элементтері арасындағы ω зейін бағаларын есептеу көрсетілген.

***

- В случае показанного выше тензора каждая строка представляет слово, 
а каждый столбец — размерность вложения
- Основная цель этого раздела — продемонстрировать, как вычисляется контекстный вектор $z^{(2)}$ с использованием  $x^{(2)}$.
- Ниже на рисунке изображён  процесса, который заключается в вычислении оценок внимания ω между $x^{(2)}$
и всеми другими входными элементами с помощью операции скалярного произведения.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="1000px">

- Біз $z^{(2)}$ контекст векторын есептеу үшін мысал ретінде $x^{(2)}$ кіріс тізбегінің 2-элементін қолданамыз; осы бөлімнің соңында біз мұны барлық контекст векторларын есептеу үшін жалпылаймыз.
- Бірінші қадам — сұраныс (query) $x^{(2)}$ және барлық басқа кіріс токендерінің арасындағы  скалярдық көбейтіндіні есептеу арқылы нормализацияланбаған зейін (attention) баллдарын есептеу:

***

- Мы используем элемент 2 входной последовательности, $x^{(2)}$, в качестве примера для вычисления контекстного вектора $z^{(2)}$; позже в этом разделе мы обобщим это для вычисления всех контекстных векторов.
- Первый шаг — вычислить ненормализованные оценки внимания (unnormalized attention scores), вычислив скалярное произведение между запросом $x^{(2)}$ и всеми остальными входными токенами:


In [59]:
import torch
import numpy as np
torch.manual_seed(42)
inputs = torch.rand(6, 3)
inputs

tensor([[0.8823, 0.9150, 0.3829],
        [0.9593, 0.3904, 0.6009],
        [0.2566, 0.7936, 0.9408],
        [0.1332, 0.9346, 0.5936],
        [0.8694, 0.5677, 0.7411],
        [0.4294, 0.8854, 0.5739]])

In [60]:
inputs = torch.tensor(
  [[0.8823, 0.9150, 0.3829], # Your     (x^1)
   [0.9593, 0.3904, 0.6009], # journey  (x^2)
   [0.2566, 0.7936, 0.9408], # starts   (x^3)
   [0.1332, 0.9346, 0.5936], # with     (x^4)
   [0.8694, 0.5677, 0.7411], # one      (x^5)
   [0.4294, 0.8854, 0.5739]] # step     (x^6)
)

**Скалярлық көбейту**  
**Скалярное произведение**

<pre>   
 [0.8823, 0.9150, 0.3829]
   X      X     X  
 [0.9593, 0.3904, 0.6009]

   ||    ||     ||

 0.8464 + 0.3572 + 0.2301 = 1.4337
</pre>
  


In [61]:
query = inputs[1]  # 2nd input token is the query
print(f"journey: {query}")

journey: tensor([0.9593, 0.3904, 0.6009])


In [62]:
attn_scores_2 = torch.empty(inputs.shape[0])
attn_scores_2

tensor([0., 0., 0., 0., 0., 0.])

In [63]:
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)
    if i < 1:
        print(f"x_{i+1}: {x_i} * query_2: {query} -> attn_scores_2: {attn_scores_2[i]}")

print(attn_scores_2)

x_1: tensor([0.8823, 0.9150, 0.3829]) * query_2: tensor([0.9593, 0.3904, 0.6009]) -> attn_scores_2: 1.4336910247802734
tensor([1.4337, 1.4337, 1.1213, 0.8493, 1.5010, 1.1024])


 - Қосымша ескерту: `скалярлық көбейту` - екі векторды элементтер бойынша көбейтуді және алынған көбейтінділерді қосуды білдіреді:
 - Примечание: `скалярное произведение` по сути представляет собой сокращённый способ записи умножения двух векторов поэлементно с последующим суммированием полученных произведений:    

 <br>

`(0.8823 * 0.9593) + (0.9150 * 0.3904) + (0.3829 * 0.6009) = 1.4337`

In [64]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(1.4337)
tensor(1.4337)


- **2-қадам:** нормаланбаған зейін баллдарын ("омегалар", ω) 1-ге тең болатындай етіп нормализациялау
- Төменде нормаланбаған зейін баллдарын қосындысы 1-ге тең болатындай нормалаудың қарапайым жолы 
(бұл – түсіндіру үшін пайдалы және модель оқыту тұрақтылығы үшін маңызды шарт):

***

- Шаг 2: нормализовать ненормализованные оценки внимания («омеги», ω) так, чтобы их сумма равнялась 1
- Ниже приведен простой способ нормализовать ненормализованные оценки внимания так, чтобы их сумма равнялась 1 (это полезно для интерпретации и важно для стабильности обучения):


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="1000px">

In [65]:
print(attn_scores_2)
print(attn_scores_2.sum())

tensor([1.4337, 1.4337, 1.1213, 0.8493, 1.5010, 1.1024])
tensor(7.4415)


In [66]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1927, 0.1927, 0.1507, 0.1141, 0.2017, 0.1481])
Sum: tensor(1.)


In [67]:
print(f"First element: {attn_scores_2[0]:.4f} / Sum: {attn_scores_2.sum():.4f} ---> {attn_scores_2[0] / attn_scores_2.sum():.4f}")

First element: 1.4337 / Sum: 7.4415 ---> 0.1927


- Алайда, тәжірибеде `softmax` функциясын нормалау үшін қолдану жиі кездеседі. Себебі, ол экстремалды мәндерді жақсы өңдейді және оқыту кезінде неғұрлым қолайлы градиенттік қасиеттерге ие.
- Төменде масштабтауға арналған `softmax` функциясының қарапайым іске асырылуы, ол сонымен қатар вектор элементтерін қосындысы 1-ге тең болатындай нормалайды:

***

- Однако на практике для нормализации обычно используется и рекомендуется функция `softmax`, которая лучше справляется с экстремальными значениями и обладает более желательными градиентными свойствами во время обучения.
-  Ниже простая  реализация функции `softmax` для масштабирования, которая также нормализует элементы вектора так, что их сумма равняется 1:

In [68]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1970, 0.1970, 0.1441, 0.1098, 0.2107, 0.1414])
Sum: tensor(1.)


- Жоғарыдағы қарапайым функция үлкен немесе кіші кіріс мәндері үшін `толып кету` `(overflow)` және `толықсымау` `(underflow)` мәселелеріне байланысты `cандық тұрақсыздық (numerical instability)` мәселелеріне ұшырауы мүмкін.
- Сондықтан, тәжірибеде өнімділік үшін жоғары оңтайландырылған `PyTorch`-тың softmax іске асырылуын қолдану ұсынылады:

***

- Вышеприведённая простая  реализация может страдать от проблем `численной нестабильности` для больших или малых входных значений из-за `переполнения (overflow)` и `андерфлоу (underflow).`
- Следовательно, на практике рекомендуется использовать вместо этого реализацию softmax из `PyTorch`, которая была высоко оптимизирована для производительности:


In [69]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1970, 0.1970, 0.1441, 0.1098, 0.2107, 0.1414])
Sum: tensor(1.0000)


- **3-қадам:** кірістің ендірілген токендерін $x^{(i)}$ зейін салмақтарына (attention weights) көбейту және алынған векторларды қосу арқылы  $z^{(2)}$ контекст векторын  есептеу:
***
- **Шаг 3:** вычислить контекстный вектор $z^{(2)}$, умножив встроенные входные токены $x^{(i)}$ на веса внимания (attention weights) и просуммировав полученные векторы:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="1000px">

In [70]:
inputs = torch.tensor(
  [[0.8823, 0.9150, 0.3829], # Your     (x^1)
   [0.9593, 0.3904, 0.6009], # journey  (x^2)
   [0.2566, 0.7936, 0.9408], # starts   (x^3)
   [0.1332, 0.9346, 0.5936], # with     (x^4)
   [0.8694, 0.5677, 0.7411], # one      (x^5)
   [0.4294, 0.8854, 0.5739]] # step     (x^6)
)

In [71]:
print("Attention weights: (a_2)", attn_weights_2)

Attention weights: (a_2) tensor([0.1970, 0.1970, 0.1441, 0.1098, 0.2107, 0.1414])


In [72]:
query = inputs[1] # 2nd input token is the query
context_vec_2 = torch.zeros(query.shape)
context_vec_2

tensor([0., 0., 0.])

In [81]:
inputs

tensor([[0.8823, 0.9150, 0.3829],
        [0.9593, 0.3904, 0.6009],
        [0.2566, 0.7936, 0.9408],
        [0.1332, 0.9346, 0.5936],
        [0.8694, 0.5677, 0.7411],
        [0.4294, 0.8854, 0.5739]])

In [82]:
attn_weights_2

tensor([0.1970, 0.1970, 0.1441, 0.1098, 0.2107, 0.1414])

In [79]:
for i,x_i in enumerate(inputs):
    print(f"{attn_weights_2[i]:.4f} * {x_i}")
    context_vec_2 += attn_weights_2[i]*x_i


print(f"context: {context_vec_2}")

0.1970 * tensor([0.8823, 0.9150, 0.3829])
0.1970 * tensor([0.9593, 0.3904, 0.6009])
0.1441 * tensor([0.2566, 0.7936, 0.9408])
0.1098 * tensor([0.1332, 0.9346, 0.5936])
0.2107 * tensor([0.8694, 0.5677, 0.7411])
0.1414 * tensor([0.4294, 0.8854, 0.5739])
context: tensor([1.9748, 2.1569, 1.8956])


In [80]:
print("Query: (x_2)", query, " ---> Context vector: (z_2)", context_vec_2)

Query: (x_2) tensor([0.9593, 0.3904, 0.6009])  ---> Context vector: (z_2) tensor([1.9748, 2.1569, 1.8956])


### 2.3.2 Барлық кіріс токендері үшін зейін салмақтарын есептеу
### 2.3.2 Вычисление весов внимания для всех входных токенов

- Жоғарыда біз 2-ші кіріс  токены үшін зейін салмақтарын және контекст векторын есептедік 
(төмендегі суретте ерекшеленген қатарда көрсетілгендей)
- Енді біз барлық зейін салмақтары мен контекст векторларын есептеу үшін бұл есептейміз

***

- Выше мы вычислили веса внимания и контекстный вектор для второго входа    
(как показано в выделенной строке на рисунке ниже)
- Далее мы обобщаем это вычисление, чтобы рассчитать все веса внимания и контекстные векторы

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="1000px">

- Зейін (self-attention) механизмінде процесс алдымен **зейін баллдарын** есептеуден басталады. Бұл баллдар кейіннен қосындысы 1-ге тең болатын **зейін салмақтарын** алу үшін нормализацияланады.
- Бұл зейін салмақтары кірістердің **салмақталған қосу** арқылы **контекст векторларын** жасау үшін қолданылады.

***

- В механизме **self-attention** процесс начинается с вычисления **оценок внимания**, которые затем нормализуются для получения **весов внимания**, в сумме дающих 1.
- Эти веса внимания затем используются для создания **контекстных векторов** путём **взвешенного суммирования** входных данных.

- Зейін баллдарын есептеу
- Вычисление оценок внимания

In [46]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[1.7623, 1.4337, 1.3128, 1.2000, 1.5703, 1.4087],
        [1.4337, 1.4337, 1.1213, 0.8493, 1.5010, 1.1024],
        [1.3128, 1.1213, 1.5807, 1.3343, 1.3708, 1.3528],
        [1.2000, 0.8493, 1.3343, 1.2436, 1.0863, 1.2254],
        [1.5703, 1.5010, 1.3708, 1.0863, 1.6274, 1.3013],
        [1.4087, 1.1024, 1.3528, 1.2254, 1.3013, 1.2977]])


- Жоғарыдағы есептеу нәтижесін матрицаларды көбейту арқылы тиімдірек қол жеткізуге болады:

- Того же результата, что и выше, можно достичь более эффективно с помощью матричного умножения:

Link: http://matrixmultiplication.xyz

In [47]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[1.7623, 1.4337, 1.3128, 1.2000, 1.5703, 1.4087],
        [1.4337, 1.4337, 1.1213, 0.8493, 1.5010, 1.1024],
        [1.3128, 1.1213, 1.5807, 1.3343, 1.3708, 1.3528],
        [1.2000, 0.8493, 1.3343, 1.2436, 1.0863, 1.2254],
        [1.5703, 1.5010, 1.3708, 1.0863, 1.6274, 1.3013],
        [1.4087, 1.1024, 1.3528, 1.2254, 1.3013, 1.2977]])


- Зейін баллдарын нормализациялау
- Нормализация оценок внимания

In [48]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2245, 0.1616, 0.1432, 0.1279, 0.1852, 0.1576],
        [0.1970, 0.1970, 0.1441, 0.1098, 0.2107, 0.1414],
        [0.1599, 0.1320, 0.2090, 0.1633, 0.1694, 0.1664],
        [0.1721, 0.1212, 0.1968, 0.1798, 0.1536, 0.1765],
        [0.1926, 0.1797, 0.1578, 0.1187, 0.2039, 0.1472],
        [0.1884, 0.1387, 0.1782, 0.1569, 0.1692, 0.1686]])


- Зейін баллдарының қосындысы 1ге тең болуын тексеру
- Проверка суммы внимания равна ли 1

In [49]:
row_2_sum = sum([0.1970, 0.1970, 0.1441, 0.1098, 0.2107, 0.1414])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- Зейін контекст векторларын есептеу
-  Вычисление контекстных векторов

In [50]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.6356, 0.7464, 0.6214],
        [0.6583, 0.7190, 0.6319],
        [0.5618, 0.7598, 0.6552],
        [0.5519, 0.7725, 0.6457],
        [0.6392, 0.7287, 0.6363],
        [0.5855, 0.7599, 0.6384]])


## 2.4 Зейін механизмін (attention mechanisms) оқытылатын салмақтармен толықтыру
## 2.4 Реализация механизма внимания (self-attention) с обучаемыми весами

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="1000px">

### 2.4.1 Зейін механизмын (attention weights) сатылап жаттықтыру
### 2.4.1 Пошаговое вычисление весов внимания (attention weights)

* Бұл бөлімде біз бастапқы трансформер архитектурасында, GPT модельдерінде және басқа да көптеген танымал Үлкен тілдік модельдерде (ҮТМ) қолданылатын **өзіндік зейін (self-attention) механизмін** іске асырамыз.
* Бұл өзіндік зейін механизмі сонымен қатар **«масштабталған скаляр көбейтінді зейіні»** (scaled dot-product attention) деп те аталады.
* Жалпы идея бұрынғыға ұқсас:
    * Біз белгілі бір кіріс элементіне тән кіріс векторлары бойынша **салмақталған қосындылар ретінде контекст векторларын** есептеу керек.
    * Ол үшін бізге **зейін салмақтары** қажет.
* Бұрын таныстырылған  зейін механизмынан аздаған айырмашылықтар бар:
    * Ең айтарлықтай айырмашылық — модельді **жаттықтыру** кезінде жаңартылатын **салмақ матрицаларының** енгізілуі.
    * Бұл жаттықтырылатын салмақ матрицалары модельдің (дәлірек айтсақ, модель ішіндегі зейін модулінің) «жақсы» контекст векторларын құрып, үйрену үшін өте маңызды.

***

* В этом разделе мы реализуем **механизм self-attention (самовнимания)**, который используется в оригинальной архитектуре трансформера, моделях GPT и большинстве других популярных больших языковых моделей (LLM).
* Этот механизм self-attention также называют **«масштабированным вниманием на основе скалярного произведения»** (scaled dot-product attention).
* Общая идея аналогична предыдущей:
    * Мы хотим вычислить **векторы контекста как взвешенные суммы** входных векторов для определённого входного элемента.
    * Для этого нам нужны **веса внимания**.
* Как вы увидите, существуют лишь незначительные отличия по сравнению с базовым механизмом внимания, представленным ранее:
    * Наиболее заметное отличие — это введение **весовых матриц**, которые обновляются во время **обучения** модели.
    * Эти обучаемые весовые матрицы крайне важны для того, чтобы модель (в частности, модуль внимания внутри модели) могла научиться создавать «хорошие» векторы контекста.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="1000px">

* Өзіндік зейін механизмін кезең-кезеңімен іске асыру үшін, біз алдымен үш жаттықтырылатын салмақ матрицасын: $W_q$, $W_k$, және $W_v$ таныстырудан бастаймыз.
* Бұл үш матрица ендірілген кіріс токендерін, $x^{(i)}$, матрицалық көбейту арқылы сұраныс (query), кілт (key) және мән (value) векторларына проекциялау үшін қолданылады:

 - Query vector: $q^{(i)} = x^{(i)}\,W_q $
 - Key vector: $k^{(i)} = x^{(i)}\,W_k $
 - Value vector: $v^{(i)} = x^{(i)}\,W_v $


***

* Реализуя механизм self-attention пошагово,   
мы начнём с введения трёх обучаемых весовых матриц: $W_q$, $W_k$, және $W_v$.

* Эти три матрицы используются для проецирования векторов-вложений (embedded) входных токенов, $x^{(i)}$, в векторы запроса (query), ключа (key) и значения (value) с помощью матричного умножения:

 - Query vector: $q^{(i)} = x^{(i)}\,W_q $
 - Key vector: $k^{(i)} = x^{(i)}\,W_k $
 - Value vector: $v^{(i)} = x^{(i)}\,W_v $

* Кіріс $x$ пен сұраныс векторының $q$ ендіру (embedding) өлшемдері модельдің дизайны мен нақты іске асырылуына байланысты бірдей немесе әртүрлі болуы мүмкін.
* GPT модельдерінде кіріс және шығыс өлшемдері әдетте бірдей болады, бірақ көрнекілік үшін, есептеуді жақсырақ бақылау мақсатында, біз бұл жерде әртүрлі кіріс және шығыс өлшемдерін таңдаймыз:

***

* Размерности вложений (embedding dimensions) для входного вектора $x$ и вектора запроса $q$ могут быть одинаковыми или разными, в зависимости от архитектуры модели и конкретной реализации.
* В моделях GPT входные и выходные размерности обычно совпадают, но в иллюстративных целях, чтобы было легче следить за вычислениями, мы выберем здесь разные входные и выходные размерности:

In [68]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

* Төменде біз үш салмақ матрицасына бастапқы мән береміз; ескеріңіз, көрнекілік мақсатында шығыс деректердегі артық ақпаратты азайту үшін біз `requires_grad=False` параметрін орнаттық. Алайда, егер біз бұл салмақ матрицаларын модельді жаттықтыру үшін қолданатын болсақ, модельді жаттықтыру кезінде осы матрицаларды жаңарту үшін `requires_grad=True` деп белгілер едік.

***

* Ниже мы инициализируем три весовые матрицы; обратите внимание, что мы устанавливаем `requires_grad=False` в иллюстративных целях, чтобы сделать вывод более наглядным. Однако, если бы мы использовали эти матрицы для обучения модели, мы бы установили `requires_grad=True`, чтобы обновлять их в процессе обучения.

In [69]:
torch.manual_seed(123)

# d_in = 3
# d_out = 2
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

* Әрі қарай сұраныс, кілт және мән векторларын есептейміз:
* Далее вычислим векторы запроса, ключа и значения:

In [70]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4268, 1.2850])


* Төменде көріп отырғанымыздай, біз 6 кіріс токенін 3D кеңістіктен 2D ендіру кеңістігіне сәтті проекцияладық:
* Как мы видим ниже, мы успешно спроецировали 6 входных токенов из 3D-пространства в 2D-пространство вложений (embedding space):

In [71]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


* 2-қадамда, біз сұраныс пен әрбір кілт векторының арасындағы скаляр көбейтіндіні есептеу арқылы нормаланбаған зейін баллдарын есептейміз:

***

* На следующем, 2-м шаге, мы вычисляем ненормализованные оценки внимания, находя скалярное произведение между вектором запроса и каждым вектором-ключом:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="1000px">

In [72]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.1888)


* Бізде 6 кіріс токен болғандықтан, берілген сұраныс векторы үшін 6 зейін балын есептейміз:

***

* Поскольку у нас 6 входов, мы получаем 6 оценок внимания для данного вектора-запроса:

In [73]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.4833, 1.1888, 1.8092, 1.5752, 1.4938, 1.5616])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="1000px">

* Содан кейін, 3-қадамда, біз бұрын қолданған `softmax` функциясы арқылы зейін салмақтарын (қосындысы 1-ге тең болатын нормаланған зейін баллдарын) есептейміз.
* Бұрынғыдан айырмашылығы, біз енді зейін баллдарын ендіру өлшемінің (embedding dimension) квадрат түбіріне, яғни $\sqrt{d_k}$ (`d_k**0.5`), бөлу арқылы масштабтаймыз:

***

* Далее, на 3-м шаге, мы вычисляем веса внимания (нормализованные оценки внимания, сумма которых равна 1), используя `softmax`-функцию, которую мы применяли ранее.
* Отличие от предыдущего раза состоит в том, что теперь мы масштабируем оценки внимания, деля их на квадратный корень из размерности вложения (embedding dimension),  
$\sqrt{d_k}$ (`d_k**0.5`):


In [74]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1612, 0.1309, 0.2030, 0.1720, 0.1624, 0.1704])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="1000px">

* **4-қадамда** біз енді 2-ші кіріс сұраныс векторы үшін контекст векторын есептейміз:

***

* На **4-м шаге** мы теперь вычисляем контекстный вектор для входного вектора-запроса 2:

In [75]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3610, 0.9555])


### 2.4.2 Шағын SelfAttention класын іске асыру
### 2.4.2 Реализация компактного класса SelfAttention

* Осының бәрін біріктіріп, біз өзіндік зейін механизмін келесідей іске асыра аламыз:

* Объединив всё вместе, мы можем реализовать механизм self-attention следующим образом:

In [76]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.3614, 0.9568],
        [0.3610, 0.9555],
        [0.3616, 0.9569],
        [0.3608, 0.9548],
        [0.3616, 0.9571],
        [0.3611, 0.9558]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="1000px">

* Жоғарыдағы іске асыруды `PyTorch`-тың `Linear` қабаттарын қолдану арқылы оңтайландыруға болады; егер ығысу бірліктерін (bias units) өшірсек, олар матрицалық көбейтуге эквивалентті болады.
* `nn.Linear`-ды біздің қолмен жасаған `nn.Parameter(torch.rand(...)` тәсілімізге қарағанда қолданудың тағы бір үлкен артықшылығы — `nn.Linear`-да салмақтарды инициализациялаудың тиімді схемасы бар, бұл модельді неғұрлым тұрақты **жаттықтыруға** әкеледі.

***

* Мы можем упростить приведенную выше реализацию, используя линейные слои (`Linear layers`) из `PyTorch`, которые эквивалентны матричному умножению, если мы отключим векторы смещения (bias units).
* Еще одно большое преимущество использования `nn.Linear` по сравнению с нашим ручным подходом `nn.Parameter(torch.rand(...)` заключается в том, что `nn.Linear` имеет предпочтительную схему инициализации весов, что приводит к более стабильному обучению модели.

In [77]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0758,  0.1225],
        [-0.0756,  0.1229],
        [-0.0742,  0.1252],
        [-0.0763,  0.1216],
        [-0.0745,  0.1247],
        [-0.0758,  0.1224]], grad_fn=<MmBackward0>)


* Ескеріңіз, `SelfAttention_v1` және `SelfAttention_v2` әртүрлі шығыс деректер3y береді, себебі олар салмақ матрицалары үшін әртүрлі бастапқы салмақтарды қолданады.
* Обратите внимание, что `SelfAttention_v1` и `SelfAttention_v2` выдают разные результаты, поскольку они используют разные начальные веса для весовых матриц.


## 2.5 Себептік (causal) зейін арқылы болашақ сөздерді жасыру
## 2.5 Скрытие будущих слов с помощью причинного (causal) внимания 

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="1000px">

### 2.5.1 Себептік (causal) зейін құрастыру
### 2.5.1 Реализация причинного  (causal) внимания

* Бұл бөлімде біз бұрын откен өзіндік зейін механизмін **себептік (causal) өзіндік зейін механизміне** түрлендіреміз.
* Себептік (causal) өзіндік зейін модельдің тізбектегі белгілі бір орынға арналған болжамының болашақ орындарға емес, тек **алдыңғы орындардағы белгілі нәтижелерге ғана тәуелді болуын** қамтамасыз етеді.
* Қарапайым сөзбен айтқанда, бұл әрбір келесі сөзді болжауды тек **алдыңғы сөздерге ғана тәуелді** болуын қамтамасыз етеді.
* Бұған қол жеткізу үшін біз әрбір берілген токен үшін **болашақ токендерді жасырамыз** (яғни, кіріс мәтінде ағымдағы токеннен кейінгі токендерді):

***

* В этом разделе мы преобразуем предыдущий механизм self-attention в **причинный (causal) механизм self-attention**.
* Причинный (causal) self-attention гарантирует, что предсказание модели для определённой позиции в последовательности будет **зависеть только от известных выходов на предыдущих позициях**, а не от будущих.
* Проще говоря, это гарантирует, что предсказание каждого следующего слова **зависит только от предшествующих слов**.
* Чтобы достичь этого, для каждого данного токена мы **маскируем будущие токены** (те, которые идут после текущего токена во входном тексте):

* **Себептік (causal) өзіндік зейінді** көрсету және іске асыру үшін, алдыңғы бөлімдегі **зейін баллдарымен** және салмақтарымен жұмыс істейміз:

***
* Чтобы проиллюстрировать и реализовать **причинное self-attention**, мы поработаем с **оценками и весами внимания** из предыдущего раздела:

In [78]:
# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1554, 0.1886, 0.1707, 0.1447, 0.1873, 0.1533],
        [0.1582, 0.1882, 0.1678, 0.1460, 0.1854, 0.1545],
        [0.1568, 0.1922, 0.1675, 0.1425, 0.1888, 0.1523],
        [0.1575, 0.1864, 0.1693, 0.1471, 0.1847, 0.1550],
        [0.1567, 0.1916, 0.1679, 0.1429, 0.1884, 0.1525],
        [0.1567, 0.1880, 0.1694, 0.1456, 0.1861, 0.1540]],
       grad_fn=<SoftmaxBackward0>)


* Болашақ зейін салмақтарын жасырудың ең қарапайым жолы — `PyTorch`-тың `tril` функциясы арқылы маска жасау. Бұл маскада негізгі диагональдан төмен орналасқан элементтер (диагональдың өзін қосқанда) 1-ге, ал негізгі диагональдан жоғары орналасқандар 0-ге теңестіріледі:
***
* Самый простой способ замаскировать будущие веса внимания — это создать маску с помощью функции `tril` из `PyTorch`, в которой элементы ниже главной диагонали (включая саму диагональ) установлены в 1, а элементы выше главной диагонали — в 0:

In [79]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [80]:
attn_weights

tensor([[0.1554, 0.1886, 0.1707, 0.1447, 0.1873, 0.1533],
        [0.1582, 0.1882, 0.1678, 0.1460, 0.1854, 0.1545],
        [0.1568, 0.1922, 0.1675, 0.1425, 0.1888, 0.1523],
        [0.1575, 0.1864, 0.1693, 0.1471, 0.1847, 0.1550],
        [0.1567, 0.1916, 0.1679, 0.1429, 0.1884, 0.1525],
        [0.1567, 0.1880, 0.1694, 0.1456, 0.1861, 0.1540]],
       grad_fn=<SoftmaxBackward0>)

* Содан кейін, диагональдан жоғары орналасқан зейін **баллдарын** нөлге айналдыру үшін біз зейін салмақтарын осы маскаға көбейтеміз:

***

* Затем мы можем умножить веса внимания на эту маску, чтобы обнулить оценки внимания над диагональю:

In [81]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1554, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1582, 0.1882, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1568, 0.1922, 0.1675, 0.0000, 0.0000, 0.0000],
        [0.1575, 0.1864, 0.1693, 0.1471, 0.0000, 0.0000],
        [0.1567, 0.1916, 0.1679, 0.1429, 0.1884, 0.0000],
        [0.1567, 0.1880, 0.1694, 0.1456, 0.1861, 0.1540]],
       grad_fn=<MulBackward0>)


* Алайда, егер маска жоғарыдағыдай `softmax`-тан кейін қолданылса, ол `softmax` арқылы жасалған ықтималдық үлестірімін бұзады.
* `Softmax` барлық шығыс мәндерінің қосындысы 1-ге тең болуын қамтамасыз етеді.
* `Softmax`-тан кейін маскалау шығыс мәндерінің қосындысы қайтадан 1-ге тең болуы үшін оларды қайта нормалауды талап етеді, бұл процесті қиындатады және күтпеген әсерлерге әкелуі мүмкін.
***
* Однако, если применять маску после `softmax`, как показано выше, это нарушит распределение вероятностей, созданное функцией `softmax`.
* `Softmax` гарантирует, что сумма всех выходных значений равна 1.
* Маскирование после `softmax` потребовало бы повторной нормализации выходов, чтобы их сумма снова стала равна 1, что усложняет процесс и может привести к нежелательным последствиям.

* Қатарлардың қосындысы 1-ге тең болуын қамтамасыз ету үшін, біз зейін салмақтарын келесідей нормалай аламыз:
***
* Чтобы убедиться, что сумма в строках равна 1, мы можем нормализовать веса внимания следующим образом:

In [82]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4567, 0.5433, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3035, 0.3722, 0.3243, 0.0000, 0.0000, 0.0000],
        [0.2385, 0.2823, 0.2563, 0.2228, 0.0000, 0.0000],
        [0.1849, 0.2260, 0.1981, 0.1687, 0.2223, 0.0000],
        [0.1567, 0.1880, 0.1694, 0.1456, 0.1861, 0.1540]],
       grad_fn=<DivBackward0>)


* Техникалық тұрғыдан себептік зейін механизмін кодтауды аяқтасақ та, жоғарыдағы нәтижеге жетудің тиімдірек тәсілін қысқаша қарастырайық.
* Сонымен, диагональдан жоғары орналасқан зейін салмақтарын нөлге айналдырып және нәтижелерді қайта нормалаудың орнына, біз диагональдан жоғары орналасқан нормаланбаған зейін **баллдарын** олар softmax функциясына кірмес бұрын теріс шексіздікпен жасыра аламыз:
***
* Хотя технически мы уже закончили с реализацией причинного механизма внимания, давайте кратко рассмотрим более эффективный подход для достижения того же результата.
* Итак, вместо того чтобы обнулять веса внимания над диагональю и повторно нормализовывать результаты, мы можем замаскировать ненормализованные оценки внимания над диагональю отрицательной бесконечностью, прежде чем они поступят в softmax-функцию:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="1000px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="1000px">

In [83]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[ 0.1371,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.0999,  0.3455,    -inf,    -inf,    -inf,    -inf],
        [ 0.1144,  0.4029,  0.2084,    -inf,    -inf,    -inf],
        [ 0.1104,  0.3489,  0.2123,  0.0139,    -inf,    -inf],
        [ 0.1163,  0.4005,  0.2143, -0.0136,  0.3769,    -inf],
        [ 0.1192,  0.3765,  0.2293,  0.0152,  0.3623,  0.0944]],
       grad_fn=<MaskedFillBackward0>)


* Төменде көріп отырғанымыздай, енді әрбір жолдағы зейін салмақтарының қосындысы қайтадан дұрыс 1-ге тең:
***
* Как мы видим ниже, теперь сумма весов внимания в каждой строке снова корректно равна 1:

In [84]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4567, 0.5433, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3035, 0.3722, 0.3243, 0.0000, 0.0000, 0.0000],
        [0.2385, 0.2823, 0.2563, 0.2228, 0.0000, 0.0000],
        [0.1849, 0.2260, 0.1981, 0.1687, 0.2223, 0.0000],
        [0.1567, 0.1880, 0.1694, 0.1456, 0.1861, 0.1540]],
       grad_fn=<SoftmaxBackward0>)


### 2.5.2 Dropout  құралы врқылы қосымша зейін салмақтарын  маскалау
### 2.5.2 Маскирование дополнительных весов внимания с помощью dropout


* Сонымен қатар, біз **жаттықтыру** кезінде артық үйренуді (overfitting) азайту үшін dropout-ты қолданамыз.
* Dropout-ты бірнеше жерде қолдануға болады:
    * мысалы, зейін салмақтарын есептегеннен кейін;
    * немесе зейін салмақтарын мән векторларына көбейткеннен кейін.
* Бұл жерде біз dropout маскасын зейін салмақтарын есептегеннен кейін қолданамыз, себебі бұл жиі кездесетін тәсіл.

* Сонымен қатар, осы нақты мысалда біз 50% dropout коэффициентін қолданамыз, бұл зейін салмақтарының жартысын кездейсоқ маскалауды білдіреді. (Кейінірек GPT моделін **жаттықтырған** кезде, біз 0.1 немесе 0.2 сияқты төменірек dropout коэффициентін қолданатын боламыз).

***

* Кроме того, мы также применяем dropout, чтобы уменьшить переобучение (overfitting) во время обучения.
* Dropout можно применять в нескольких местах:
    * например, после вычисления весов внимания;
    * или после умножения весов внимания на векторы-значения.
* Здесь мы будем применять маску dropout после вычисления весов внимания, поскольку это более распространённый подход.

* Более того, в этом конкретном примере мы используем коэффициент dropout 50%, что означает случайное маскирование половины весов внимания. (Когда мы будем обучать модель GPT позже, мы будем использовать более низкий коэффициент dropout, например, 0.1 или 0.2).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="1000px">

* Егер біз 0.5 (50%) dropout коэффициентін қолдансақ, сақталған (жойылмаған) мәндер сәйкесінше 1/0.5 = 2 коэффициентіне масштабталады.
* Масштабтау `1 / (1 - dropout_rate)` формуласы бойынша есептеледі.
***
* Если мы применим коэффициент dropout 0.5 (50%), то неотброшенные значения будут соответственно отмасштабированы на коэффициент 1/0.5 = 2.
* Масштабирование вычисляется по формуле: `1 / (1 - dropout_rate)`.

In [85]:
example = torch.ones(6, 6)
example

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [86]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [87]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6070, 0.7443, 0.6487, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5647, 0.5127, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4521, 0.0000, 0.3373, 0.0000, 0.0000],
        [0.0000, 0.3760, 0.3389, 0.2913, 0.3723, 0.0000]],
       grad_fn=<MulBackward0>)


### 2.5.3 Шағын себептік (causal) өзіндік зейін (self-attention) класын іске асыру
### 2.5.3 Реализация компактного класса причинного (causal)  механизма внимания (self-attention)


* Енді біз себептік және dropout маскаларын қамтитын, өзіндік зейін механизмінің жұмыс істейтін нұсқасын іске асыруға дайынбыз.
* `CausalAttention` класымыз 2-тарауда іске асырған деректерді жүктеуші (data loader) шығаратын батч (топтама) шығыстарын қолдауы үшін, бірнеше кірістен тұратын батчтарды (топтамаларды) өңдейтін кодты іске асыруымыз керек.
* Қарапайымдылық үшін, осындай батчтық кірісті модельдеу мақсатында біз кіріс мәтінінің мысалын қайталаймыз:
***
* Теперь мы готовы реализовать рабочую версию self-attention, включающую причинную маску и маску dropout.
* Ещё один момент: необходимо реализовать код для обработки батчей, состоящих из нескольких входов, чтобы наш класс `CausalAttention` поддерживал батчевые выходы, генерируемые загрузчиком данных (data loader), который мы реализовали в главе 2.
* Для простоты, чтобы симулировать такой батчевый вход, мы дублируем пример входного текста:

In [88]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [89]:
batch

tensor([[[0.8823, 0.9150, 0.3829],
         [0.9593, 0.3904, 0.6009],
         [0.2566, 0.7936, 0.9408],
         [0.1332, 0.9346, 0.5936],
         [0.8694, 0.5677, 0.7411],
         [0.4294, 0.8854, 0.5739]],

        [[0.8823, 0.9150, 0.3829],
         [0.9593, 0.3904, 0.6009],
         [0.2566, 0.7936, 0.9408],
         [0.1332, 0.9346, 0.5936],
         [0.8694, 0.5677, 0.7411],
         [0.4294, 0.8854, 0.5739]]])

In [90]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method. 
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.8340, -0.3584],
         [-0.7842, -0.2061],
         [-0.7248, -0.1467],
         [-0.6725, -0.1607],
         [-0.6983, -0.1409],
         [-0.6864, -0.1554]],

        [[-0.8340, -0.3584],
         [-0.7842, -0.2061],
         [-0.7248, -0.1467],
         [-0.6725, -0.1607],
         [-0.6983, -0.1409],
         [-0.6864, -0.1554]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


* Ескеріңіз, dropout тек **жаттықтыру** кезінде ғана қолданылады, инференс (генерация) кезінде емес.
***
* Обратите внимание, что dropout применяется только во время обучения, а не во время инференса (генерации).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="1000px">

## 2.6 Жалғыс басты зейін механизмды көп басты зейін механизмына өзгерту 
## 2.6 Расширение одноголового механизма внимания до многоголового механизма внимания

### 2.6.1 Бірнеше жалғыс басты зейін механизмдерды бірге қосу
### 2.6.1 Наложение нескольких слоев одноголового внимания

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="1000px">

* Көп басты зейін модулін алу үшін біз бірнеше бір басты зейін модульдерін жай ғана қабаттаймыз:
***
* Мы просто накладываем несколько модулей одноголового внимания, чтобы получить модуль многоголового внимания:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="1000px">

* Көп басты зейіннің негізгі идеясы – зейін механизмін әртүрлі, үйретілген сызықтық проекциялармен бірнеше рет (параллельді түрде) іске қосу. Бұл модельге әртүрлі позициялардағы әртүрлі репрезентация кеңістіктерінен алынған ақпаратқа бірлесіп назар аударуға мүмкіндік береді.
***
Основная идея многоголового внимания заключается в том, чтобы запускать механизм внимания несколько раз (параллельно) с разными, обученными линейными проекциями. Это позволяет модели совместно обращать внимание на информацию из разных подпространств представлений на разных позициях.

In [91]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.8340, -0.3584,  0.7834,  0.6197],
         [-0.7842, -0.2061,  0.7593,  0.5032],
         [-0.7248, -0.1467,  0.7082,  0.4678],
         [-0.6725, -0.1607,  0.6466,  0.4565],
         [-0.6983, -0.1409,  0.6726,  0.4510],
         [-0.6864, -0.1554,  0.6590,  0.4557]],

        [[-0.8340, -0.3584,  0.7834,  0.6197],
         [-0.7842, -0.2061,  0.7593,  0.5032],
         [-0.7248, -0.1467,  0.7082,  0.4678],
         [-0.6725, -0.1607,  0.6466,  0.4565],
         [-0.6983, -0.1409,  0.6726,  0.4510],
         [-0.6864, -0.1554,  0.6590,  0.4557]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- Жоғарыдағы іске асыруда ендіру өлшемі (embedding  ) 4-ке тең, 
өйткені біз кілт (query), сұраныс (key), мән (value) векторлары    
және контекст (context) векторы үшін ендіру өлшемі ретінде `d_out=2` мәнін қолдандық.    
Бізде 2 зейін басы болғандықтан, шығыс ендіру өлшемі 2*2=4 құрайды.
***
- В реализации выше размерность эмбеддинга равна 4,   
потому что мы используем `d_out=2` как размерность эмбеддинга для векторов ключа, запроса и значения,    
а также для контекстного вектора. И поскольку у нас 2 головы внимания, выходная размерность эмбеддинга составляет 2*2=4.

In [92]:
heads = nn.ModuleList([
CausalAttention(d_in, d_out, context_length, 0.0), 
CausalAttention(d_in, d_out, context_length, 0.0)
])

### 2.6.2 Салмақтарды бөлу арқылы көп басты зейін механизмын (self-attention) іске асыру
### 2.6.2 Реализация многоголового механизма внимания(self-attention) с разделением весов

- Жоғарыдағы код көп басты зейіннің механизмын интуитивті іске асыруы болғанымен (бұрынғы `CausalAttention` бір басты зейін механизмдерді біріктіру арқылы), дәл соған қол жеткізу үшін біз `MultiHeadAttention` деп аталатын жеке класс жаза аламыз.
- Біз `MultiHeadAttention` класы үшін біз бір басты зейін механизмдерді бастарын біріктірмейміз.
- Оның орнына, біз бірыңғай `W_query`, `W_key` және `W_value` салмақ матрицаларын жасаймыз да, содан кейін оларды әрбір зейін басы үшін жеке матрицаларға бөлеміз:
***
- Хотя вышеописанное является интуитивно понятной и полнофункциональной реализацией многоголового внимания (обертывающей ранее созданную реализацию одноголового внимания `CausalAttention`), мы можем написать отдельный класс под названием `MultiHeadAttention` для достижения того же результата.
- Для этого отдельного класса `MultiHeadAttention` мы не конкатенируем отдельные головы внимания.
- Вместо этого мы создаем единые весовые матрицы `W_query`, `W_key` и `W_value`, 
а затем разделяем их на отдельные матрицы для каждой головы внимания:

In [93]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)  
        self.register_buffer(                                          
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.2511, 0.2183],
         [0.2784, 0.2632],
         [0.2814, 0.3009],
         [0.2694, 0.3244],
         [0.2780, 0.3152],
         [0.2728, 0.3183]],

        [[0.2511, 0.2183],
         [0.2784, 0.2632],
         [0.2814, 0.3009],
         [0.2694, 0.3244],
         [0.2780, 0.3152],
         [0.2728, 0.3183]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- Ескеріңіз, жоғарыда көрсетілген нұсқа `MultiHeadAttentionWrapper`-дың қайта жазылған нұсқасы болып табылады.
- Кездейсоқ салмақ инициализациялары әртүрлі болғандықтан, нәтижелік шығыс сәл өзгеше көрінеді, бірақ екеуі де алдағы тарауларда іске асыратын GPT класында қолдануға болатын толық функционалды іске асырулар болып табылады.
***
- Обратите внимание, что вышеприведенное, по сути, является переписанной  версией `MultiHeadAttentionWrapper`.
- Итоговый результат выглядит немного иначе, поскольку различаются случайные инициализации весов, но оба варианта являются полностью функциональными реализациями, которые можно использовать в классе GPT, который мы будем реализовывать в следующих главах.

**Шығыс өлшемдері туралы ескертпе**
- Жоғарыдағы `MultiHeadAttention`-да  бұрынғы `MultiHeadAttentionWrapper` класындағыдай параметрді пайдалану үшін `d_out=2` деп қойдым.
- `MultiHeadAttentionWrapper` конкатенацияға байланысты шығыс басының өлшемін `d_out * num_heads` (яғни, `2*2 = 4`) ретінде қайтарады.
- Алайда, `MultiHeadAttention` класы (оны пайдаланушыға ыңғайлы ету үшін) шығыс басының өлшемін тікелей `d_out` арқылы басқаруға мүмкіндік береді; бұл дегеніміз, егер біз `d_out = 2` деп қойсақ, шығыс басының өлшемі бастар санына қарамастан 2 болады.
***
**Примечание о выходных размерностях**
- В приведённом выше `MultiHeadAttention` я использовал `d_out=2`, чтобы применить ту же настройку, что и в классе `MultiHeadAttentionWrapper` ранее.
- `MultiHeadAttentionWrapper` из-за конкатенации возвращает выходную размерность головы `d_out * num_heads` (т.е., `2*2 = 4`).
- Однако класс `MultiHeadAttention` (чтобы сделать его более удобным для пользователя) позволяет нам напрямую контролировать выходную размерность головы через `d_out`; это означает, что если мы установим `d_out = 2`, выходная размерность головы будет равна 2, независимо от количества голов.

- Ескеріңіз, сонымен қатар біз жоғарыдағы `MultiHeadAttention` класына сызықтық проекция қабатын (`self.out_proj`) қостық. Бұл – өлшемдерді өзгертпейтін қарапайым сызықтық түрлендіру. LLM іске асыруларында мұндай проекция қабатын қолдану стандартты тәжірибе болып саналады, бірақ ол қатаң түрде міндетті емес.
***
- Обратите внимание, что мы также добавили слой линейной проекции (`self.out_proj`) в приведённый выше класс `MultiHeadAttention`. Это простое линейное преобразование, которое не изменяет размерности. Использование такого проекционного слоя является стандартной практикой при реализации LLM, но оно не является строго необходимым.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="1000px">

- Ескеріңіз, егер сізді жоғарыдағының ықшам әрі тиімді іске асырылуы қызықтырса, PyTorch-тағы `torch.nn.MultiheadAttention` класын да қарастыруыңызға болады.
***
- Обратите внимание, что если вас интересует компактная и эффективная реализация вышеизложенного, вы также можете рассмотреть класс `torch.nn.MultiheadAttention` в PyTorch.

- `MultiheadAttention` механизмысындағы зейін баллдарының кадам кадаммен есептеу әдісі
- Пошаговое вычисление баллов внимания в `MultiheadAttention` механизме

In [94]:
W_key = nn.Linear(d_in, d_out, bias=False)
keys = W_key(inputs)
keys

tensor([[-0.1608,  0.5821],
        [-0.3328,  0.2560],
        [-0.1634, -0.0290],
        [-0.0162,  0.1699],
        [-0.3129,  0.2260],
        [-0.0973,  0.2799]], grad_fn=<MmBackward0>)

In [95]:
inputs

tensor([[0.8823, 0.9150, 0.3829],
        [0.9593, 0.3904, 0.6009],
        [0.2566, 0.7936, 0.9408],
        [0.1332, 0.9346, 0.5936],
        [0.8694, 0.5677, 0.7411],
        [0.4294, 0.8854, 0.5739]])

In [96]:
inputs.shape

torch.Size([6, 3])

In [97]:
keys.shape

torch.Size([6, 2])

In [98]:
keys

tensor([[-0.1608,  0.5821],
        [-0.3328,  0.2560],
        [-0.1634, -0.0290],
        [-0.0162,  0.1699],
        [-0.3129,  0.2260],
        [-0.0973,  0.2799]], grad_fn=<MmBackward0>)

In [99]:
b=1
num_tokens =6   
num_heads =2
head_dim =1
keys2 = keys.view(b, num_tokens, num_heads, head_dim) 

In [100]:
keys2.shape

torch.Size([1, 6, 2, 1])

In [101]:
keys2

tensor([[[[-0.1608],
          [ 0.5821]],

         [[-0.3328],
          [ 0.2560]],

         [[-0.1634],
          [-0.0290]],

         [[-0.0162],
          [ 0.1699]],

         [[-0.3129],
          [ 0.2260]],

         [[-0.0973],
          [ 0.2799]]]], grad_fn=<ViewBackward0>)

In [102]:
keys2[:, :, 0, :]

tensor([[[-0.1608],
         [-0.3328],
         [-0.1634],
         [-0.0162],
         [-0.3129],
         [-0.0973]]], grad_fn=<SliceBackward0>)

In [103]:
keys2[:, :, 1, :]

tensor([[[ 0.5821],
         [ 0.2560],
         [-0.0290],
         [ 0.1699],
         [ 0.2260],
         [ 0.2799]]], grad_fn=<SliceBackward0>)

In [104]:
keys

tensor([[-0.1608,  0.5821],
        [-0.3328,  0.2560],
        [-0.1634, -0.0290],
        [-0.0162,  0.1699],
        [-0.3129,  0.2260],
        [-0.0973,  0.2799]], grad_fn=<MmBackward0>)

In [105]:
keys3 = keys2.transpose(1, 2)
keys3.shape

torch.Size([1, 2, 6, 1])

In [106]:
keys3

tensor([[[[-0.1608],
          [-0.3328],
          [-0.1634],
          [-0.0162],
          [-0.3129],
          [-0.0973]],

         [[ 0.5821],
          [ 0.2560],
          [-0.0290],
          [ 0.1699],
          [ 0.2260],
          [ 0.2799]]]], grad_fn=<TransposeBackward0>)

In [107]:
keys3.transpose(2, 3)

tensor([[[[-0.1608, -0.3328, -0.1634, -0.0162, -0.3129, -0.0973]],

         [[ 0.5821,  0.2560, -0.0290,  0.1699,  0.2260,  0.2799]]]],
       grad_fn=<TransposeBackward0>)

In [108]:
W_query = nn.Linear(d_in, d_out, bias=False)
queries = W_query(inputs)
queries = queries.view(b, num_tokens, num_heads, head_dim)
queries = queries.transpose(1, 2)
attn_scores = queries @ keys3.transpose(2, 3) 